In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

# os.environ['GITHUB_APP_ID'] = os.getenv('GITHUB_APP_ID', '')
# os.environ['GITHUB_APP_PRIVATE_KEY'] = os.getenv('GITHUB_APP_PRIVATE_KEY', '')
# os.environ['GITHUB_REPOSITORY'] = os.getenv('GITHUB_REPOSITORY', '')
# os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', '')


True

In [10]:
# Initialize tools first
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit

github_api = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github_api)
github_tools = toolkit.get_tools()
github_tools

[GitHubAction(name='Get Issues', description="\nThis tool will fetch a list of the repository's issues. It will return the title, and issue number of 5 issues. It takes no input.", args_schema=<class 'langchain_community.agent_toolkits.github.toolkit.NoInput'>, api_wrapper=GitHubAPIWrapper(github=<github.MainClass.Github object at 0x7fd174316570>, github_repo_instance=Repository(full_name="BENHIMA-Mohamed-Amine/full-guide-for-my-github"), github_repository='BENHIMA-Mohamed-Amine/full-guide-for-my-github', github_app_id='1472101', github_app_private_key='../secrets/github-app-private-key.pem', active_branch='main', github_base_branch='main'), mode='get_issues'),
 GitHubAction(name='Get Issue', description='\nThis tool will fetch the title, body, and comment thread of a specific issue. **VERY IMPORTANT**: You must specify the issue number as an integer.', args_schema=<class 'langchain_community.agent_toolkits.github.toolkit.GetIssue'>, api_wrapper=GitHubAPIWrapper(github=<github.MainClas

In [11]:
from langchain_core.tools import tool
from typing import Dict

# Global variables to store GitHub tools
read_file_tool = None
overview_tool = None

def init_github_tools(github_tools):
    """Initialize GitHub tools from toolkit"""
    global read_file_tool, overview_tool
    
    read_file_tool = next((tool for tool in github_tools if tool.name == "Read File"), None)
    overview_tool = next((tool for tool in github_tools if tool.name == "Overview of existing files in Main branch"), None)
    
    if not read_file_tool or not overview_tool:
        raise ValueError("Required GitHub tools not found")

@tool
def read_readme_file() -> str:
    """
    Read the README.md file content from the repository.
    Returns the complete README content as a string.
    """
    if read_file_tool is None:
        return "Read File tool is not initialized. Please initialize GitHub tools first."
    try:
        readme_content = read_file_tool.invoke({ 
            "formatted_filepath": "README.md"
        })
        
        # Check if the result is an error message
        if "File not found" in readme_content or "404" in readme_content:
            return "README.md file not found in the repository"
        
        return readme_content
    except Exception as e:
        return f"Error reading README.md: {str(e)}"

@tool 
def get_project_overview() -> str:
    """
    Get an overview of all files in the main branch of the repository.
    Returns a structured view of the project's file organization with the project title.
    """
    if overview_tool is None:
        return "Overview tool is not initialized. Please initialize GitHub tools first."
    try:
        overview = overview_tool.invoke({})
        return overview + f"\n the project title is {os.getenv('GITHUB_REPOSITORY', 'Unknown Project')}"
    except Exception as e:
        return f"Error getting project overview: {str(e)}"

@tool
def detect_tech_stack() -> Dict[str, str] | str:
    """
    Read dependency files to detect technology stack.
    Returns a dictionary with file names as keys and their content as values.
    """
    tech_stack_files = {}
    
    # Common dependency files to check
    dependency_files = [
        "package.json",
        "requirements.txt", 
        "Cargo.toml",
        "pom.xml",
        "build.gradle",
        "go.mod",
        "composer.json",
        "Gemfile",
        "poetry.lock",
        "yarn.lock",
        "Pipfile"
    ]
    if read_file_tool is None:
        return "Read File tool is not initialized. Please initialize GitHub tools first."
    
    for file_name in dependency_files:
        try:
            content = read_file_tool.invoke({
                "formatted_filepath": file_name
            })
            
            # Check if the result is an error message
            if "File not found" in content or "404" in content:
                # Skip files that don't exist
                continue
                
            tech_stack_files[file_name] = content
        except Exception:
            # Skip if there's any other error
            continue
    
    return tech_stack_files 

# Tools list for LangGraph
tools = [detect_tech_stack, read_readme_file, get_project_overview]

In [12]:

# Initialize the custom tools
init_github_tools(github_tools)

# Now use the tools
readme_content = read_readme_file.invoke({})
overview = get_project_overview.invoke({})
tech_files = detect_tech_stack.invoke({})

print("Tech stack files found:", list(tech_files.keys()))

Tech stack files found: []


In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
MODEL_NAME = "gemini-2.0-flash"
model = ChatGoogleGenerativeAI(model=MODEL_NAME)
model

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7fd16c75ba40>, default_metadata=(), model_kwargs={})

In [14]:
LINKEDIN_POST_PROMPT = """You are a professional LinkedIn content creator specializing in software development and tech showcase posts.

Your task is to create a compelling LinkedIn post that showcases a GitHub project in a professional, engaging manner.

## AVAILABLE TOOLS:
- read_readme_file(): Get README content
- get_project_overview(): Get file structure overview  
- detect_tech_stack(): Get technology stack from dependency files
- get_repository_metadata(): Get repo name, description, etc.
- get_recent_commits(): Get recent development activity

## CONTEXT GATHERING:
1. First, gather comprehensive information about the project:
   - Repository name and description
   - Technology stack and dependencies
   - Key features from README
   - Project structure and organization
   - Recent development activity

## LINKEDIN POST REQUIREMENTS:

### STRUCTURE:
1. **Hook** (1-2 lines): Engaging opening that captures attention
2. **Project Introduction** (2-3 lines): What the project does and why it matters
3. **Technical Highlights** (3-4 lines): Key technologies and interesting implementation details
4. **Key Features** (2-3 lines): Main functionalities and benefits
5. **Call-to-Action** (1-2 lines): Encourage engagement (check it out, feedback, etc.)
6. **Hashtags** (5-8 relevant hashtags)

### TONE & STYLE:
- Professional but approachable
- Focus on business value, not just technical details
- Use emojis sparingly (1-2 max)
- Write in first person
- Keep sentences concise and scannable
- Avoid overly technical jargon

### FORMATTING:
- Use line breaks for readability
- Bold key points with **text**
- Use bullet points if listing features
- Keep total length under 1300 characters
- Make it mobile-friendly

### CONTENT GUIDELINES:
- Highlight the problem the project solves
- Mention the tech stack naturally
- Include any interesting challenges overcome
- Show passion for the technology
- Make it relevant to your professional network

### HASHTAG STRATEGY:
- Include technology-specific tags (#Python, #React, etc.)
- Add industry tags (#SoftwareDevelopment, #AI, etc.)
- Include community tags (#OpenSource, #GitHub, etc.)
- Add personal branding tags (#BuildingInPublic, #TechInnovation)

## EXAMPLE OUTPUT FORMAT:
Just shipped a new [project type] that [key benefit]! 🚀
[Project name] is a [brief description] built to [solve what problem].
Tech Stack: [main technologies] on form of bullet points and the purpose of each one.
Key Features:

- [Feature 1]
- [Feature 2]
- [Feature 3]

Check it out on GitHub and let me know what you think! Always open to feedback and collaboration.
#TechTag1 #TechTag2 #SoftwareDevelopment #OpenSource #BuildingInPublic #GitHub #TechInnovation #Programming

## INSTRUCTIONS:
1. Use ALL available tools to gather comprehensive project information
2. Analyze the gathered data to understand the project's value proposition
3. Create a LinkedIn post following the structure and guidelines above (you can enhance it with your own insights but keep it short and to the point)
4. Ensure the post is engaging, professional, and showcases the project effectively
5. Adapt the tone based on the project type (e.g., more technical for dev tools, more business-focused for applications)

Begin by gathering all project information using the available tools, then create the LinkedIn post.
"""

In [15]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()

# control iteration loop
max_iterations = 7
recursion_limit = 2 * max_iterations + 1

agent = create_react_agent(
    model=model,
    tools=tools,
    prompt=LINKEDIN_POST_PROMPT,
    checkpointer=memory,
    debug=True,
).with_config(recursion_limit=recursion_limit)

In [16]:
config = {"configurable": {"thread_id": "linkedin-agent-1"}}

In [17]:
response = agent.invoke(
    {"messages": [("user", "Create a professional LinkedIn post showcasing this GitHub repository")]}, 
    config # type: ignore
)

[values] {'messages': [HumanMessage(content='Create a professional LinkedIn post showcasing this GitHub repository', additional_kwargs={}, response_metadata={}, id='66b5c9f3-d388-441d-89c9-21260ae77d34')]}
[updates] {'agent': {'messages': [AIMessage(content="Okay, I'm ready to create a LinkedIn post. First, I need to gather information about the project using the available tools. I'll start by reading the README file, getting the project overview, and detecting the tech stack.", additional_kwargs={'function_call': {'name': 'detect_tech_stack', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--be2d5cc8-e323-4fda-9753-0c36d124a3c7-0', tool_calls=[{'name': 'read_readme_file', 'args': {}, 'id': 'a655ec97-781c-410e-91ca-4db748d45d9a', 'type': 'tool_call'}, {'name': 'get_project_overview', 'args': {}, 'id': '13d39679-1a46-4d7a-8507-76f9c1afa4b9', 'typ

In [22]:
from IPython.display import display, Markdown
display(Markdown(response['messages'][-1].content))

Navigating my GitHub can be a maze! 🧭 That's why I created a central guide to showcase my projects.
This repo, **full-guide-for-my-github**, provides a curated list of links to my various projects, making it easier to explore my work.
It covers areas like:
*   **GenAI**: Agentic coding, website summarization
*   **NLP**: Translation apps, sentiment analysis
*   **CV**: Plant disease detection
*   **ML**: Classification models
*   **Web Dev**: Stock management website

Check it out to see my work across different domains! Open to connections and feedback.
#GitHub #OpenSource #Portfolio #SoftwareDevelopment #MachineLearning #NLP #GenAI #WebDev